In [53]:
import numpy as np
import drone
import importlib
import plotly
import quaternion

o = importlib.reload(drone);
o = importlib.reload(quaternion);
plotly.offline.init_notebook_mode(connected=True)


In [63]:

def run_sim(u, dt, z0):
    
    # initialize vectors
    s = np.empty((u.shape[0], 14))
    s[:] = np.nan  
    e = np.empty((u.shape[0], 3))
    e[:] = np.nan  

    # define initial conditions
    s[0, :] = np.zeros(14)
    s[0, 2] = z0
    s[0, 3] = 1.

    # run sim
    for i in range(1, u.shape[0]):
        s[i, :] = drone.step_quadcopter(s[i-1, :], u[i-1, :], dt)
        e[i, :] = quaternion.to_euler(s[i, 3:7])

    return s, e


def plot_result(t, s, e, title):
          
    data_1 = [
        plotly.graph_objs.Scatter(x=t, y=s[:, 0].squeeze(), name="X"),
        plotly.graph_objs.Scatter(x=t, y=s[:, 1].squeeze(), name="Y"),
        plotly.graph_objs.Scatter(x=t, y=s[:, 2].squeeze(), name="Z"),
    ]
    
    data_2 = [
        plotly.graph_objs.Scatter(x=t, y=e[:, 0].squeeze(), name="Roll"),
        plotly.graph_objs.Scatter(x=t, y=e[:, 1].squeeze(), name="Pitch"),
        plotly.graph_objs.Scatter(x=t, y=e[:, 2].squeeze(), name="Yaw"),
    ]
    
    data_3 = [
        plotly.graph_objs.Scatter(x=t, y=s[:, 4].squeeze(), name="Q_i"),
        plotly.graph_objs.Scatter(x=t, y=s[:, 5].squeeze(), name="Q_j"),
        plotly.graph_objs.Scatter(x=t, y=s[:, 6].squeeze(), name="Q_k"),
    ]
    
    fig = plotly.subplots.make_subplots(rows=1, cols=3)
    for d in data_1:
        fig.add_trace(d, row=1, col=1)
    for d in data_2:
        fig.add_trace(d, row=1, col=2)
    for d in data_3:
        fig.add_trace(d, row=1, col=3)
        
    plotly.offline.plot(fig, filename=title + ".html")


In [64]:

# length and timestep of simulation
n_samples = 500
dt = 0.01
t = np.arange(0, n_samples * dt, dt)

# define input vector
u = np.linspace(0, 6, n_samples).reshape(-1, 1) @ np.ones(4).reshape(1, -1)
u = 3 * np.ones((n_samples, 4))
u[3000:, :] = 0.

# vary forward motors 
u[:, 0] = u[:, 0] + 0.05 * np.sin(2 * np.pi * t)
u[:, 1] = u[:, 1] + 0.05 * np.sin(2 * np.pi * t)

s, e = run_sim(u, dt, z0=5.)


In [65]:

plot_result(t, s, e, "pitch_test")


In [66]:

# length and timestep of simulation
n_samples = 500
dt = 0.01
t = np.arange(0, n_samples * dt, dt)

# define input vector
u = np.linspace(0, 6, n_samples).reshape(-1, 1) @ np.ones(4).reshape(1, -1)
u = 3 * np.ones((n_samples, 4))
u[3000:, :] = 0.

# vary left hand side motors 
u[:, 0] = u[:, 0] + 0.05 * np.sin(2 * np.pi * t)
u[:, 2] = u[:, 2] + 0.05 * np.sin(2 * np.pi * t)

s, e = run_sim(u, dt, z0=5.)


In [67]:

plot_result(t, s, e, "roll_test")


In [68]:

# length and timestep of simulation
n_samples = 500
dt = 0.01
t = np.arange(0, n_samples * dt, dt)

# define input vector
u = np.linspace(0, 6, n_samples).reshape(-1, 1) @ np.ones(4).reshape(1, -1)
u = 3 * np.ones((n_samples, 4))
u[3000:, :] = 0.

# vary diagonal motors 
u[:, 0] = u[:, 0] + 0.05 * np.sin(2 * np.pi * t)
u[:, 3] = u[:, 3] + 0.05 * np.sin(2 * np.pi * t)

s, e = run_sim(u, dt, z0=5.)


In [69]:

plot_result(t, s, e, "yaw_test")
